In [1]:
!gsutil cp gs://chunkerbert/v2/docker.txt .
!gsutil cp gs://chunkerbert/v2/windows.txt .

Copying gs://chunkerbert/v2/docker.txt...
\ [1 files][723.2 MiB/723.2 MiB]                                                
Operation completed over 1 objects/723.2 MiB.                                    
Copying gs://chunkerbert/v2/windows.txt...
| [1 files][877.7 MiB/877.7 MiB]                                                
Operation completed over 1 objects/877.7 MiB.                                    


In [2]:
!gsutil cp gs://chunkerbert/v2/bert-wordpiecev3-vocab.txt .

Copying gs://chunkerbert/v2/bert-wordpiecev2-vocab.txt...
/ [1 files][  1.2 MiB/  1.2 MiB]                                                
Operation completed over 1 objects/1.2 MiB.                                      


In [4]:
!rm dataset.txt
!cat docker.txt >> dataset.txt
!cat windows.txt >> dataset.txt
!ls -lah dataset.txt

-rw-r--r-- 1 jupyter jupyter 1.6G Aug 19 13:16 dataset.txt


In [5]:
!pip install -q sentencepiece 
!pip install -q nltk
!git clone https://github.com/google-research/bert

fatal: destination path 'bert' already exists and is not an empty directory.


In [6]:
import os
import sys
import json
import nltk
import random
import logging
import tensorflow as tf
import sentencepiece as spm

from glob import glob
from tensorflow.keras.utils import Progbar

sys.path.append("bert")

from bert import modeling, optimization, tokenization
from bert.run_pretraining import input_fn_builder, model_fn_builder

  
# configure logging
log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s :  %(message)s')
sh = logging.StreamHandler()
sh.setLevel(logging.INFO)
sh.setFormatter(formatter)
log.handlers = [sh]


In [7]:
regex_tokenizer = nltk.RegexpTokenizer("\w+")

def normalize_text(text):
  # lowercase text
  text = str(text).lower()
  # remove non-UTF
  text = text.encode("utf-8", "ignore").decode()
  # remove punktuation symbols
  text = " ".join(regex_tokenizer.tokenize(text))
  return text

def count_lines(filename):
  count = 0
  with open(filename) as fi:
    for line in fi:
      count += 1
  return count

In [8]:
RAW_DATA_FPATH = "dataset.txt" #@param {type: "string"}
PRC_DATA_FPATH = "proc_dataset.txt" #@param {type: "string"}

# apply normalization to the dataset
# this will take a minute or two

total_lines = count_lines(RAW_DATA_FPATH)
bar = Progbar(total_lines)

with open(RAW_DATA_FPATH,encoding="utf-8") as fi:
  with open(PRC_DATA_FPATH, "w",encoding="utf-8") as fo:
    for l in fi:
      fo.write(normalize_text(l)+"\n")
      bar.add(1)

726164/726164 [==============================] - 25s 35us/step


In [9]:
VOC_FNAME = "bert-wordpiecev3-vocab.txt" #@param {type:"string"}

In [10]:
!rm -rf ./shards
!mkdir ./shards
!split -a 4 -l 25600 -d $PRC_DATA_FPATH ./shards/shard_
!ls ./shards/

shard_0000  shard_0005	shard_0010  shard_0015	shard_0020  shard_0025
shard_0001  shard_0006	shard_0011  shard_0016	shard_0021  shard_0026
shard_0002  shard_0007	shard_0012  shard_0017	shard_0022  shard_0027
shard_0003  shard_0008	shard_0013  shard_0018	shard_0023  shard_0028
shard_0004  shard_0009	shard_0014  shard_0019	shard_0024


In [11]:
MAX_SEQ_LENGTH = 512 #@param {type:"integer"}
MASKED_LM_PROB = 0.15 #@param
MAX_PREDICTIONS = 77 #@param {type:"integer"}
DO_LOWER_CASE = True #@param {type:"boolean"}
PROCESSES = 6 #@param {type:"integer"}
PRETRAINING_DIR = "pretraining_data_docker4" #@param {type:"string"}
BUCKET_NAME = "chunkbert_training" #@param {type:"string"}

In [12]:
XARGS_CMD = ("ls ./shards/ | "
             "xargs -n 1 -P {} -I{} "
             "python3 bert/create_pretraining_data.py "
             "--input_file=./shards/{} "
             "--output_file={}/{}.tfrecord "
             "--vocab_file={} "
             "--do_lower_case={} "
             "--max_predictions_per_seq={} "
             "--max_seq_length={} "
             "--masked_lm_prob={} "
             "--random_seed=34 "
             "--dupe_factor=5")
XARGS_CMD = XARGS_CMD.format(PROCESSES, '{}', '{}', PRETRAINING_DIR, '{}', 
                             VOC_FNAME, DO_LOWER_CASE, 
                             MAX_PREDICTIONS, MAX_SEQ_LENGTH, MASKED_LM_PROB)

In [9]:
tf.gfile.MkDir(PRETRAINING_DIR)
!$XARGS_CMD

2022-08-11 21:25:57,063 :  From /tmp/ipykernel_1485/312981126.py:1: The name tf.gfile.MkDir is deprecated. Please use tf.io.gfile.mkdir instead.





W0811 21:25:58.990888 140667984258880 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0811 21:25:58.991050 140667984258880 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0811 21:25:58.991186 140667984258880 module_wrapper.py:139] From /home/jupyter/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



W0811 21:25:59.000122 139675671770944 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0811 21:25:59.000275 139675671770944 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0811 21:25:

In [ ]:
!gsutil -m cp -r $PRETRAINING_DIR gs://$BUCKET_NAME